In [3]:
from pyspark.sql import SparkSession, functions as F, types as T

In [4]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [5]:
print(spark.version)

3.5.1


In [71]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [72]:
df_joined = film_category_df.join(category_df, on="category_id", how="left")
df_grouped = df_joined.groupBy("name").count()
df_ordered_group = df_grouped.orderBy(F.col("count").desc()).show()

+-----------+-----+
|       name|count|
+-----------+-----+
|     Sports|   74|
|    Foreign|   73|
|     Family|   69|
|Documentary|   68|
|  Animation|   66|
|     Action|   64|
|        New|   63|
|      Drama|   62|
|      Games|   61|
|     Sci-Fi|   61|
|   Children|   60|
|     Comedy|   58|
|     Travel|   57|
|   Classics|   57|
|     Horror|   56|
|      Music|   51|
+-----------+-----+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [73]:
actor_df = actor_df.withColumn("full_name", F.concat_ws(" ", actor_df.first_name, actor_df.last_name))

df_joined = film_actor_df.join(actor_df, on="actor_id", how="left")\
                            .join(film_df, on="film_id", how="left")\
                            .join(inventory_df, on="film_id", how="left")\
                            .join(rental_df, on="inventory_id", how="left")
df_filtered = df_joined.filter(F.col("rental_id").isNotNull())
df_grouped = df_filtered.groupBy("full_name").count()
df_ordered_group = df_grouped.orderBy(F.col("count").desc()).show(10)


+------------------+-----+
|         full_name|count|
+------------------+-----+
|       SUSAN DAVIS|  825|
|    GINA DEGENERES|  753|
|    MATTHEW CARREY|  678|
|       MARY KEITEL|  674|
|ANGELA WITHERSPOON|  654|
|       WALTER TORN|  640|
|       HENRY BERRY|  612|
|       JAYNE NOLTE|  611|
|        VAL BOLGER|  605|
|     SANDRA KILMER|  604|
+------------------+-----+
only showing top 10 rows



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [74]:
category_df = category_df.withColumnRenamed("name", "category_name")
df_joined = payment_df.join(rental_df, on="rental_id", how="left")\
                            .join(inventory_df, on="inventory_id", how="left")\
                            .join(film_df, on="film_id", how="left")\
                            .join(film_category_df, on="film_id", how="left")\
                            .join(category_df, on="category_id", how="left")

df_grouped = df_joined.groupBy("category_name").agg(F.sum("amount").alias("total_amount"))
df_ordered_group = df_grouped.orderBy(F.col("total_amount").desc()).show(1)

+-------------+-----------------+
|category_name|     total_amount|
+-------------+-----------------+
|       Sports|5314.209999999847|
+-------------+-----------------+
only showing top 1 row



4.
Вивести назви фільмів, яких не має в inventory.

In [75]:
df_not_in_inventory = film_df.join(inventory_df, on="film_id", how="left_anti")
df_not_in_inventory.select("title").show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [76]:
actor_df = actor_df.withColumn("full_name", F.concat_ws(" ", actor_df.first_name, actor_df.last_name))
category_df = category_df.withColumnRenamed("name", "category_name")

df_joined = film_actor_df.join(actor_df, on="actor_id", how="left")\
                            .join(film_df, on="film_id", how="left")\
                            .join(film_category_df, on="film_id", how="left")\
                            .join(category_df, on="category_id", how="left")

df_filtered = df_joined.filter(F.col("category_name").isin(["Children"]))
df_grouped = df_filtered.groupBy("full_name").count()
df_ordered_group = df_grouped.orderBy(F.col("count").desc()).show(3)


+------------+-----+
|   full_name|count|
+------------+-----+
|HELEN VOIGHT|    7|
| SUSAN DAVIS|    6|
|  MARY TANDY|    5|
+------------+-----+
only showing top 3 rows



Stop Spark session:

In [77]:
spark.stop()